In [1]:
import random
import timeit
import numpy
from numpy.random import seed
from tensorflow.python.framework.random_seed import set_seed
import os
from tqdm import tqdm
import tensorflow as tf
from  functools import reduce
from keras.layers import Dense,Flatten, Dropout
from  keras import Input, Model
import keras.backend as K

import pandas as pd
from ast import literal_eval
from sklearn.model_selection import train_test_split, KFold
import numpy as np
from itertools import chain
import en_core_web_sm

from transformers import AutoConfig, AutoModel,AutoTokenizer
from transformers import DistilBertTokenizer, TFDistilBertModel
from keras.layers import Dense,Flatten, Dropout
from  keras import Input, Model
import keras.backend as K
from transformers import TFBertModel
from transformers import TFRobertaModel,  TFAlbertModel, TFXLNetModel

import tensorflow as tf
from keras import Input, Model
from keras.layers import Dense, Flatten, Layer
from transformers import TFBertModel


nlp = en_core_web_sm.load()

seed(1)
set_seed(2)
from pandas import *

In [2]:
url='/kaggle/input/toxicspan/CR_full_span_dataset.csv'
full_token='/kaggle/input/toxicspan/full_token_list.csv'

In [3]:
##test train ratio
def generate_test_train_index( length, ratio):
    rand_array=np.arange(0,length,1)
    train, test =train_test_split(rand_array,test_size=ratio, shuffle=True )
    train=numpy.sort(train)
    test=numpy.sort(test)
    return train, test

In [4]:
# spliting each text (Code Rreview comment) to sentences
def split_paragraph_to_sentences(data):
    base_offset = []
    sentences = []
    sentence_spans = []
    sent_start = []

    print("Shape: ", data.shape[0])

    for i in range(len(data)):
        # print( data['text'][i])
        text = str(data['text'][i])
        spans = data['spans'][i]
        left_ptr = 0
        right_ptr = 0
        started = False
        doc = nlp(text)
        for sent in doc.sents:
            start = doc[sent.start].idx
            end = doc[sent.end - 1].idx + len(doc[sent.end - 1])
            if end - start <= 1:
                continue
            sentences.append(text[start:end])
            base_offset.append(start)
            if started == False:
                started = True
                sent_start.append(True)
            else:
                sent_start.append(False)
            while left_ptr < len(spans) and spans[left_ptr] < start:
                left_ptr += 1
            right_ptr = left_ptr
            cur_spans = []
            while right_ptr < len(spans) and spans[right_ptr] < end:
                cur_spans.append(spans[right_ptr])
                right_ptr += 1
            sentence_spans.append(cur_spans)
            left_ptr = right_ptr
        if started == False:
            sentences.append(text)
            base_offset.append(0)
            sentence_spans.append(spans)
            sent_start.append(True)
    data_processed = pd.DataFrame(
        data={"text": sentences, "spans": sentence_spans, "base_offset": base_offset, "sent_start": sent_start})
    return data_processed # dataframe -> 

#    text                            spans                 base_offset   sent_start
# 0  "yeah that sucked, fixed   [10, 11, 12, 13, 14, 15]         0         True

# base_offset: starting index of the sentence in the original text
# sent_start: Boolean flag indicating if the sentence is the start of a new sentence in the paragraph.

In [5]:
# prepare the output toxic/non toxic labels
# purpose : checking number of toxic characters is greater than or equal to half the length of the token
def generate_output(data, tokens, max_length,tokenizer): #((dataframe->'text','spans','base_offset','sent_start'), (input_ids,token_type_ids,attention_mask,special_tokens_mask),70,bert) 
        output = np.zeros((data.shape[0], max_length)) # np.zeros((1, 70))
        for i in range(data.shape[0]): # data.shape[0]=1
            isToxic = np.zeros(len(data.text[i])) # len=22
            # np.zeros(len("yeah that sucked, fixed"))
            # np.zeros(22)
            #is_toxic=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,0.])
            for idx in data.spans[i]: # data.spans[0]: [10, 11, 12, 13, 14, 15] 
                isToxic[idx - data.base_offset[i]] = 1 # toxic letter=1  // base_offset[0]=0
#idx = 10
#isToxic[10 - 0] = 1
# isToxic[10]=1
# array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

#final isToxic=[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.]

            # finding special tokens
            for j in range(max_length): # 0-69
                if tokens.special_tokens_mask[i][j]: # [0][0] i=dataframe row / j=every row 70 array elements/// check if thele element is special token
                    continue

                sp_tok=tokenizer.convert_ids_to_tokens(tokens.input_ids[i][j]) 
                # tokenizer.convert_ids_to_tokens converts the token ID back to its string representation
                # If the token ID 1037 corresponds to the word "sucked", then convert_ids_to_tokens will return "sucked"
                
                ##this portion is added to making some unwanted tokens as 0 (non-toxic)
                # these tokens are generated for pretrained tokenizers
                if(sp_tok=="_" or sp_tok=="s" or sp_tok=="D"):
                    continue
                if (sp_tok == "Ġ" or  sp_tok=="Ġ," or sp_tok=="Ġ." or sp_tok=="Ġ/" or sp_tok=="Ġs" or sp_tok=="Ġ[" or sp_tok=="Ġy" or sp_tok=="Ġd" or sp_tok=="Ġ@" or sp_tok=="Ġa"):
                    continue
                if (sp_tok=="ĠI" or sp_tok=="Ġ(" or sp_tok=="Ġi" or sp_tok=="Ġ*" or sp_tok=="Ġ_" or sp_tok=="Ġ:" or sp_tok=="Ġ=" or sp_tok=="Ġ-"):
                    continue
                if(sp_tok=="ĠU" or sp_tok=="Ġ#" or sp_tok=="ĠD" or sp_tok=="ĠA" or sp_tok=="ĠS" or sp_tok=="Ġ\"" or sp_tok=="Ġ," or sp_tok=="Ġ>"):
                    continue
                start = tokens.offset_mapping[i][j][0] # i= sentence number // j= token number // 0=start 1=end
                end = tokens.offset_mapping[i][j][1]
                cnt = 0
                for pos in range(start, end):
                    if isToxic[pos]:
                        cnt += 1
# Check if the number of toxic characters (cnt) is greater than or equal to half the length of the token.                       
                if cnt >= (end - start + 1) // 2:
                    output[i][j] = 1
        return output  # returns array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


In [6]:
##preprocess each sentence
def sentence_preprocessing(data, max_length, tokenizer):    #dataframe,70,bert
        data_processed = split_paragraph_to_sentences(data) # in: dataframe / return: 'text': ["yeah that sucked, fixed"], 'spans': [[10, 11, 12, 13, 14, 15]], 'base_offset': [0], 'sent_start': [True]
        print(len(data_processed['text'])) # 1
        input_texts =data_processed['text'] #"  pd.Series(["yeah that sucked, fixed"])
        input_texts=input_texts.to_numpy() # np.array(["yeah that sucked, fixed"]) ::::: converted to numpy array->Performance Optimization,Compatibility,Memory Usage
        tokens = tokenizer(list(data_processed.text), max_length=max_length, padding="max_length",         
                           truncation=True, return_offsets_mapping=True, return_special_tokens_mask=True,
                           return_token_type_ids=True)

# tokens = {
#     "input_ids": [ 101, 2748, 2008, 5638, 1010, 4714, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ],
    # Each unique word or subword has a specific ID in the tokenizer's vocabulary.
#     "token_type_ids": [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ],               
    # Segment IDs (0 indicates all tokens are in the same segment/sentence)
#     "attention_mask": [ 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ],               
    # Attention mask (1 means the token should be attended to) (1=words 0=padding)
#     "special_tokens_mask": [ 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ],           
    # Special tokens mask (1 for special tokens like [CLS] , [SEP] and [PAD]) (1 for special tokens, 0 for regular tokens)
    # "offset_mapping": [ (0, 0), (0, 4), (5, 9), (10, 16), (16, 17), (18, 23), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), 
    #                    (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0) ]
    # every word's start end position
# }



        data_input = {"input_ids": np.array(tokens.input_ids),
                      "token_type_ids": np.array(tokens.token_type_ids),
                      "attention_mask": np.array(tokens.attention_mask)}


# data_input = {
# "input_ids": np.array([[101, 2748, 2008, 5638, 1010, 4714, 102]]),
# "token_type_ids": np.array([[0, 0, 0, 0, 0, 0, 0]]),
# "attention_mask": np.array([[1, 1, 1, 1, 1, 1, 1]])
# }

        # to check whether contains toxic word
        data_output = generate_output(data_processed, tokens, max_length,tokenizer) 
# np.array([[0, 0, 0, 1, 0, 0, 0]])  =  ((dataframe->'text','spans','base_offset','sent_start'), (input_ids,token_type_ids,attention_mask,special_tokens_mask),70,bert)   

        inital_weights = 1 - np.array(tokens.special_tokens_mask) # 1-[1,0,0,0,0,0,1]=np.array([[0, 1, 1, 1, 1, 1, 0]])
        return data_input, input_texts, data_output, inital_weights,tokens


# Data Input: {'input_ids': array([[101, 2026, 2001, 2005, 1037, 3946, 102]]),
#              'token_type_ids': array([[0, 0, 0, 0, 0, 0, 0]]),
#              'attention_mask': array([[1, 1, 1, 1, 1, 1, 1]])}

# Input Texts: ['yeah that sucked, fixed']

# Data Output: [[0, 0, 0, 1, 0, 0, 0]]

# Initial Weights: [[0, 1, 1, 1, 1, 1, 0]]

# Tokens: {'input_ids': [[101, 2026, 2001, 2005, 1037, 3946, 102]],
#          'token_type_ids': [[0, 0, 0, 0, 0, 0, 0]],
#          'attention_mask': [[1, 1, 1, 1, 1, 1, 1]],
#          'special_tokens_mask': [[1, 0, 0, 0, 0, 0, 1]]
#          "offset_mapping": [(0, 0), (0, 4), (5, 9), (10, 16), (16, 17), (18, 23), (0, 0)] 
#}


In [7]:
def classify_by_threshold(prediction, thresh):
  predictions = np.zeros(prediction.shape)
  for i in range(prediction.shape[0]):
    for j in range(prediction.shape[1]):
      if prediction[i,j]>=thresh:
        predictions[i,j]=1
  return predictions


In [8]:
## this function is for text encoding which provides the explainable output
def convert_to_original_text_with_toxicity(encoded_text,pred,tokenizer):
  ##list_decoded_text_toxicity = []
  only_decoded_text = tokenizer.convert_ids_to_tokens(encoded_text)
  decode = [tokenizer.convert_tokens_to_string([i]) for i in tokenizer.convert_ids_to_tokens(encoded_text)]
  decoded_new = []
  prev = 0

  if (len(pred) == 0):
    decoded_new = decode
  else:
    #if (pred[0] == 0):
      #decoded_new.append(['<toxic>'])
    for i in range(0, len(pred)):
      pred_val = int(pred[i])

      ##for handling exception
      if (pred_val >= len(decode)):
        print("Missing here", only_decoded_text, pred)
        break
      if (pred_val > prev and prev > 0):
        decoded_new.append(['</toxic>'])
      decoded_new.append(decode[prev:pred_val])

      if (pred_val - prev > 0):
        decoded_new.append(['<toxic>'])

      decoded_new.append([decode[pred_val]])

      #decoded_new.append([decode[pred_val]])
      # if(pred_val - prev >0): decoded_new.append(['</toxic>'])
      prev = pred_val + 1
    decoded_new.append(['</toxic>'])
    decoded_new.append(decode[prev:len(decode)])

  decoded_new = list(chain.from_iterable(decoded_new))
  return decoded_new


In [9]:
## if we want to use BIO for prediction
# we do not use it in our experiment
def classify_bio_output(prediction):
  num_samples = prediction.shape[0]*prediction.shape[1]
  predictions = np.zeros((prediction.shape[0],prediction.shape[1]))
  for i in range(prediction.shape[0]):
    for j in range(prediction.shape[1]):
      if prediction[i,j] != 0:
        predictions[i,j] = 1
  return predictions

In [10]:
# class ReshapeLayer(Layer):
#     def __init__(self, target_shape, **kwargs):
#         super(ReshapeLayer, self).__init__(**kwargs)
#         self.target_shape = target_shape

#     def call(self, inputs):
#         return tf.reshape(inputs, self.target_shape)



In [11]:
# import tensorflow as tf
# from tensorflow.keras.layers import Layer, Input, Dense, Conv1D, GlobalMaxPooling1D, Dropout
# from tensorflow.keras.models import Model
# from transformers import TFBertModel, BertConfig

# class BertLayer(Layer):
#     def __init__(self, model_name, training=True, **kwargs):
#         super(BertLayer, self).__init__(**kwargs)
#         self.training = training
#         config = BertConfig()  # Create a configuration with default settings
#         self.bert = TFBertModel(config)

#     def call(self, inputs, training=None):
#         input_ids = inputs["input_ids"]
#         token_type_ids = inputs["token_type_ids"]
#         attention_mask = inputs["attention_mask"]

#         outputs = self.bert(
#             input_ids=input_ids,
#             token_type_ids=token_type_ids,
#             attention_mask=attention_mask,
#             training=self.training,
#         )
#         return outputs.last_hidden_state  # Return embeddings (last hidden state)


In [12]:
# def get_bert(training, max_length=70):
#     # Define Keras inputs
#     input_ids = Input(shape=(max_length,), name="input_ids", dtype="int32")
#     token_type_ids = Input(shape=(max_length,), name="token_type_ids", dtype="int32")
#     attention_mask = Input(shape=(max_length,), name="attention_mask", dtype="int32")

#     # BERT embeddings with defined embedding dimension
#     embedding_dim = 768  # Default for BERT-base
#     bert_outputs = BertLayer(model_name="bert-base-uncased", training=training)(
#         {"input_ids": input_ids, "token_type_ids": token_type_ids, "attention_mask": attention_mask}
#     )
#     bert_outputs = ReshapeLayer((-1, max_length, embedding_dim))(bert_outputs)
#     print("BERT output shape:", bert_outputs.shape)

#     # Sequential CNN layers for feature extraction
#     conv_1 = Conv1D(filters=512, kernel_size=5, activation="relu")(bert_outputs)
#     conv_1 = Dropout(0.5)(conv_1)
#     print("Conv1 output shape:", conv_1.shape)

#     conv_2 = Conv1D(filters=256, kernel_size=5, activation="relu")(conv_1)
#     conv_2 = Dropout(0.5)(conv_2)
#     print("Conv2 output shape:", conv_2.shape)

#     conv_3 = Conv1D(filters=128, kernel_size=3, activation="relu")(conv_2)
#     conv_3 = Dropout(0.5)(conv_3)
#     print("Conv3 output shape:", conv_3.shape)

#     conv_4 = Conv1D(filters=64, kernel_size=3, activation="relu")(conv_3)
#     conv_4 = Dropout(0.5)(conv_4)
#     pooled_4 = GlobalMaxPooling1D()(conv_4)
#     print("Pooled output shape:", pooled_4.shape)

#     # Dense classification head with additional layers and dropout
#     dense_layer = Dense(128, activation="relu")(pooled_4)
#     dense_layer = Dropout(0.5)(dense_layer)
#     dense_layer = Dense(64, activation="relu")(dense_layer)
#     dense_layer = Dropout(0.5)(dense_layer)
#     outputs = Dense(1, activation="sigmoid")(dense_layer)

#     # Define the model
#     return Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=outputs)




# def get_roberta(training, max_length=70):
#   input_ids = Input(shape=(max_length,),name="input_ids",dtype="int32")
#   token_type_ids = Input(shape=(max_length,),name="token_type_ids")
#   attention_mask = Input(shape=(max_length,),name="attention_mask")
    

#   # Convert KerasTensors to Tensors 
#   input_ids = tf.convert_to_tensor(input_ids)
#   token_type_ids = tf.convert_to_tensor(token_type_ids) 
#   attention_mask = tf.convert_to_tensor(attention_mask)
    
#   encoder = TFRobertaModel.from_pretrained("roberta-base")

#   embeddings = encoder({"input_ids":input_ids,"token_type_ids":token_type_ids,"attention_mask":attention_mask}, training=training)[0]

#   dense_layer2=Dense(1,activation='sigmoid') (embeddings, training=training)
#   outputs = Flatten()(dense_layer2)
#   return Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=outputs)

In [13]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Layer
from transformers import TFRobertaModel

class MyLayer(Layer):
    def __init__(self, **kwargs):
        super(MyLayer, self).__init__(**kwargs)
        self.dense_layer = Dense(1, activation='sigmoid')

    def build(self, input_shape):
        self.encoder = TFRobertaModel.from_pretrained("roberta-base")
        super(MyLayer, self).build(input_shape)

    def call(self, inputs):
        input_ids, token_type_ids, attention_mask = inputs
        # Convert token_type_ids to int32
        token_type_ids = tf.cast(token_type_ids, dtype=tf.int32)
        embeddings = self.encoder({"input_ids": input_ids, "token_type_ids": token_type_ids, "attention_mask": attention_mask}, training=True)[0]
        outputs = self.dense_layer(embeddings)
        outputs = Flatten()(outputs)
        return outputs

    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], 1)  # Assuming the input shape is (batch_size, max_length)

def get_roberta(training, max_length=70):
    input_ids = Input(shape=(max_length,), name="input_ids", dtype="int32")
    token_type_ids = Input(shape=(max_length,), name="token_type_ids")
    attention_mask = Input(shape=(max_length,), name="attention_mask")
    
    outputs = MyLayer()([input_ids, token_type_ids, attention_mask])
    return tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=outputs)


In [14]:
import numpy as np

def offset_precision_recall_fscore(gold, predictions):
    num_correct = len(set(predictions).intersection(set(gold)))
    num_predicted = len(set(predictions))
    correct_span_length = len(set(gold))

    if num_predicted == 0:
        if correct_span_length == 0:
            precision = 1  # non-toxic instance correctly predicted
        else:
            precision = 0
    else:
        precision = num_correct / num_predicted

    if correct_span_length == 0:
        if num_predicted == 0:
            recall = 1  # non-toxic correctly predicted
            f1 = 1  # non-toxic correctly predicted
        else:
            recall = 0
            f1 = 0
    else:
        recall = num_correct / correct_span_length
        f1 = (2 * num_correct) / (correct_span_length + num_predicted)
    
    return precision, recall, f1



In [15]:
def compute_performance(ground_truth, predicted_output, test_texts, tokenizer, maxlen, error_analysis=False):
    instance_f1_0 = []
    instance_prec_0 = []
    instance_recall_0 = []
    instance_f1_1 = []
    instance_prec_1 = []
    instance_recall_1 = []
    retro_output = []

    for i in range(len(ground_truth)):
        pred_offset = []
        test_base_offset = []

        for j in range(maxlen):
            if ground_truth[i][j] == 1:
                test_base_offset.append(j)  # ground truth offset
            if predicted_output[i][0] >= 0.5:  # Assuming threshold of 0.5 for binary classification
                pred_offset.append(j)  # token offset for prediction

        if len(test_base_offset) == 0:  # non-toxic instance
            precision_0, recall_0, f1_0 = offset_precision_recall_fscore(test_base_offset, pred_offset)
            instance_prec_0.append(precision_0)
            instance_recall_0.append(recall_0)
            instance_f1_0.append(f1_0)
        else:  # toxic instances
            precision_1, recall_1, f1_1 = offset_precision_recall_fscore(test_base_offset, pred_offset)
            instance_prec_1.append(precision_1)
            instance_recall_1.append(recall_1)
            instance_f1_1.append(f1_1)
        
        if error_analysis:
            gold = '[' + ','.join(str(v) for v in test_base_offset) + ']'
            predicted = '[' + ','.join(str(w) for w in pred_offset) + ']'
            original_text = test_texts[i]

            list_decoded_text_toxicity = []
            only_decoded_text = []

            if len(original_text) == 0:
                only_decoded_text = []
            else:
                encoded_text = tokenizer.encode(original_text)
                only_decoded_text = tokenizer.convert_ids_to_tokens(encoded_text)
                decoded_actual_text = [tokenizer.convert_tokens_to_string([i]) for i in tokenizer.convert_ids_to_tokens(encoded_text)]

                if len(pred_offset) == 0:
                    list_decoded_text_toxicity = decoded_actual_text
                else:
                    list_decoded_text_toxicity = convert_to_original_text_with_toxicity(encoded_text, pred_offset, tokenizer)

                only_decoded_text_str = ' '.join(str(x) for x in only_decoded_text)
                decoded_text_toxicity_to_string = ' '.join(str(y) for y in list_decoded_text_toxicity)
                list_decoded_text_toxicity_to_string = [decoded_text_toxicity_to_string]

            retro_output.append([original_text, list_decoded_text_toxicity_to_string, gold, predicted])

    avg_precision_0 = np.mean(np.array(instance_prec_0))
    avg_recall_0 = np.mean(np.array(instance_recall_0))
    avg_f1_0 = np.mean(np.array(instance_f1_0))

    avg_precision_1 = np.mean(np.array(instance_prec_1))
    avg_recall_1 = np.mean(np.array(instance_recall_1))
    avg_f1_1 = np.mean(np.array(instance_f1_1))

    return avg_precision_0, avg_recall_0, avg_f1_0, avg_precision_1, avg_recall_1, avg_f1_1, retro_output


In [16]:
##collecting the pretrained tokenizer
def tokenizer_collect(Tokenizer):
    Tokenizer=Tokenizer.lower()
    if (Tokenizer == 'bert'):
        return AutoTokenizer.from_pretrained("bert-base-uncased")
    elif (Tokenizer == 'roberta'):
        return AutoTokenizer.from_pretrained("roberta-base",add_prefix_space=True)
    elif (Tokenizer == 'dbert'):
        return AutoTokenizer.from_pretrained("distilbert-base-uncased")
    elif (Tokenizer == 'albert'):
        return AutoTokenizer.from_pretrained("albert-base-v2")
    elif (Tokenizer == 'deberta'):
        return AutoTokenizer.from_pretrained("kamalkraj/deberta-base")
    elif (Tokenizer == 'xlnet'):
        return AutoTokenizer.from_pretrained("xlnet-base-cased")
    else:
        print("Wrong tokenizer selected")

In [17]:
import tensorflow.keras.backend as K
# def binary_loss(y_true, y_pred):
#   loss = -1 * y_true * K.log( K.clip(y_pred + K.epsilon(), 0, 1.0) )
#   loss += -1 * (1-y_true) * K.log( K.clip(1-y_pred+K.epsilon(), 0, 1.0) )
#   return loss

# import tensorflow as tf
# from tensorflow.keras import backend as K

# def binary_loss(y_true, y_pred):
#     # Clip the predictions to avoid log(0) errors
#     y_pred = tf.clip_by_value(y_pred, K.epsilon(), 1.0 - K.epsilon())
#     # Compute binary cross-entropy loss
#     loss = -y_true * tf.math.log(y_pred) - (1 - y_true) * tf.math.log(1 - y_pred)
#     return tf.reduce_mean(loss)



# def binary_loss(y_true, y_pred):
#     # Clipping y_pred to ensure it stays within (0, 1)
#     y_pred = K.clip(y_pred, K.epsilon(), 1.0 - K.epsilon())
#     loss = -1 * y_true * K.log(y_pred)
#     loss += -1 * (1 - y_true) * K.log(1 - y_pred)
#     return K.mean(loss)

## we used only binary_loss() for our experiment
def binary_loss(y_true, y_pred):
  loss = -1 * y_true * K.log( K.clip(y_pred + K.epsilon(), 0, 1.0) )
  loss += -1 * (1-y_true) * K.log( K.clip(1-y_pred+K.epsilon(), 0, 1.0) )
  return loss

def weighted_mse(y_true, y_pred):
    mse= tf.keras.losses.MeanSquaredError()
    loss =mse(y_true, y_pred).numpy()
    return loss


## this is for a full token list with 239 tokens
## these tokens are used to add with pre-trained tokenizers
## after trained with the tokens, pretrained tokenizers can provide a full word tokenization and improvs expalinability


new_token_data = read_csv(full_token)     #("models/full_token_list.csv")
new_token_list=new_token_data['words'].tolist()


In [18]:
class ToxiSpanSE:
    def __init__(self, ALGO="ROBERTA", Tokenizer="roberta",
                 model_file=url, #"models/CR_full_span_dataset.xlsx",
                 load_pretrained=False, max_len=70):
        self.classifier_model = None
        self.modelFile = model_file
        self.Tokenizer=Tokenizer
        self.ALGO = ALGO
        self.data = self._read_data_from_file(model_file) # return full dataframe
        self.load_pretrained = load_pretrained
        self.max_length=max_len

    def _read_data_from_file(self, model_file):
        dataframe =pd.read_csv(model_file)
        #dataframe=dataframe.iloc[:8]
        dataframe.sample(frac=1).reset_index(drop=True) # call a random shuffle
        #print(dataframe)
        return  dataframe

    ##get the DL models
    def get_model(self, params, max_length):
        ALGO = self.ALGO.upper()
        training = params['training']
        if(ALGO=="BERT"):
            return get_bert(training, max_length)
        elif(ALGO=="DBERT"):
            return get_distilbert(training, max_length)
        elif (ALGO == "ALBERT"):
            return get_albert(training, max_length)
        elif (ALGO == "ROBERTA"):
            return get_roberta(training, max_length)
        elif (ALGO == "XLNET"):
            return get_xlnet(training, max_length)
        else:
            print("Unknown algorithm: " + ALGO)
            exit(1)

    ##define tokenizer
    def get_tokenizer(self):
        tokenizer = tokenizer_collect(self.Tokenizer) #bert
        return tokenizer #bert


    def get_training_data(self):
        return self.data  # full dataframe




In [19]:
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
import numpy as np
import timeit
import pandas as pd
from ast import literal_eval

# Initialize parameters
rand_state = 42
max_length = 70
threshold = None #.32
toxicClassifier = ToxiSpanSE()

# Prepare dataset
dataset = toxicClassifier.get_training_data() #dataframe
if threshold:
    threshold = float(threshold)
random_folding = KFold(n_splits=6, shuffle=True, random_state=rand_state)

dataset["spans"] = dataset["spans"].apply(literal_eval) #list [10, 11, 12, 13, 14, 15]
tokenizer = toxicClassifier.get_tokenizer() # bert

# Add custom tokens (optional)
# tokenizer.add_tokens(new_token_list)

data_input, input_texts, data_output, initial_weights, tokens = sentence_preprocessing(
    dataset, max_length, tokenizer     # dataframe,70,bert
)
# print("data output start")
# print(data_output)
# print("data output end")
# Data Input: {'input_ids': array([[101, 2026, 2001, 2005, 1037, 3946, 102]]),
#              'token_type_ids': array([[0, 0, 0, 0, 0, 0, 0]]),
#              'attention_mask': array([[1, 1, 1, 1, 1, 1, 1]])}

# Input Texts: ['yeah that sucked, fixed']

# Data Output: [[0, 0, 0, 1, 0, 0, 0]]

# Initial Weights: [[0, 1, 1, 1, 1, 1, 0]]

# Tokens: {'input_ids': [[101, 2026, 2001, 2005, 1037, 3946, 102]],
#          'token_type_ids': [[0, 0, 0, 0, 0, 0, 0]],
#          'attention_mask': [[1, 1, 1, 1, 1, 1, 1]],
#          'special_tokens_mask': [[1, 0, 0, 0, 0, 0, 1]]
#          "offset_mapping": [(0, 0), (0, 4), (5, 9), (10, 16), (16, 17), (18, 23), (0, 0)] 
#}

# Debugging: Check the data shapes
print(f"Data Input Shape: {data_input['input_ids'].shape}") # (70,)
print(f"Data Output Shape: {data_output.shape}") # (1, 70)

# Identify toxic words for validation
toxic_words = []
for i in range(data_output.shape[0]): # 1
    for j in range(max_length): # 70
        if data_output[i, j] == 1 and initial_weights[i, j] == 1:
            toxic_words.append(tokenizer.convert_ids_to_tokens(tokens.input_ids[i][j])) # converted to string

# Debugging toxic words (optional)
# toxic_words_freq = pd.Series(toxic_words).value_counts()
# print(toxic_words_freq[:15])

# Start 10-fold CV
count = 1
error_analysis_output = []
results = ""



Shape:  19651
34600
Data Input Shape: (34600, 70)
Data Output Shape: (34600, 70)


In [20]:
def partition_input_dict(inputDict, dataIndex):
    # print("dataindex start")
    # print(dataIndex)
    # print("dataindex end")
    returnDict = dict()
    returnDict["input_ids"] = inputDict["input_ids"][dataIndex]
    returnDict["token_type_ids"] = inputDict["token_type_ids"][dataIndex]
    returnDict["attention_mask"] = inputDict["attention_mask"][dataIndex]
    #print(f"returnDict["input_ids"]:{returnDict["input_ids"] }")
    # print("return dict start")
    # print(returnDict["input_ids"])
    # print("return dict end")
    return returnDict


In [ ]:
for train_index, test_index in random_folding.split(data_output): # Data Output: [[0, 0, 0, 1, 0, 0, 0]]    
    start = timeit.default_timer()
    print(f"Using split-{count} as test data...")

    # print("train_index:{train_index}")
    # print("test_index:{test_index}")
    # print("data_output:{data_output}")

    # print("train_index start")
    # print(train_index)
    # print("train_index end")

    # print("test_index start")
    # print(test_index)
    # print("test_index end")
    
    # Split data
    train_input_set = partition_input_dict(data_input, train_index)
    # print("train_input_set start")
    # print(train_input_set)
    # print("train_input_set end") 
    
# Data Input: {'input_ids': array([[101, 2026, 2001, 2005, 1037, 3946, 102]]),
#              'token_type_ids': array([[0, 0, 0, 0, 0, 0, 0]]),
#              'attention_mask': array([[1, 1, 1, 1, 1, 1, 1]])}
# train_index: [[0, 0, 0, 1, 0, 0, 0]]  
    
    test_input = partition_input_dict(data_input, test_index)
    
    test_texts = input_texts[test_index]
    train_output = data_output[train_index]
    test_output = data_output[test_index]

    params = {
        "training": True,
        "learning_rate": 1e-5,
        "pre_training_epoch": 0,
        "initial_epoch": 0,
        "epochs":30,
        "silver": False,
    }

    training_index, validation_index = generate_test_train_index(train_output.shape[0], 0.1)
    # print("training_index start")
    # print(training_index)
    # print("training_index end") 

    # print("validation_index start")
    # print(validation_index)
    # print("validation_index end") 
    
    val_texts = input_texts[validation_index]
    # print("val_texts start")
    # print(val_texts)
    # print("val_texts end") 
    model_training_input = partition_input_dict(train_input_set, training_index)
    # print("model_training_input start")
    # print(model_training_input)
    # print("model_training_input end") 
    validation_input = partition_input_dict(train_input_set, validation_index)
    # print("validation_input start")
    # print(validation_input)
    # print("validation_input end") 
    model_training_output = train_output[training_index]
    # print("model_training_output start")
    # print(model_training_output)
    # print("model_training_output end") 
    validation_output = train_output[validation_index]
    # print("validation_output start")
    # print(validation_output)
    # print("validation_output end") 

    # Build and compile the model
    model = toxicClassifier.get_model(params, max_length)
    optimizer = tf.keras.optimizers.Adam(learning_rate=params["learning_rate"])
    model.compile(optimizer=optimizer, loss=binary_loss)
    model.summary()

    es_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)
    sample_weight = initial_weights[training_index]

    # Train the model
    print("Training the model...")
    model.fit(
        model_training_input,
        model_training_output,
        epochs=params["epochs"],
        initial_epoch=params['initial_epoch'],
        validation_data=(validation_input, validation_output),
        sample_weight=sample_weight,
        callbacks=[es_callback],
    )

    # Debugging: Check training input/output shapes
    print(f"Training Input Shape: {model_training_input['input_ids'].shape}")
    print(f"Validation Input Shape: {validation_input['input_ids'].shape}")

    # Predict on test data
    test_pred = model.predict(test_input, verbose=1, batch_size=64)
    
    print("test_pred start")
    print(test_pred)
    print("test_pred end") 
    
    stop = timeit.default_timer()
    time_elapsed = stop - start
    print(f"Time elapsed for split-{count}: {time_elapsed} seconds")

    # Debugging: Inspect predictions
    print(f"Sample test predictions: {test_pred[:10]}")
    print(f"Min prediction: {np.min(test_pred)}, Max prediction: {np.max(test_pred)}")

    
    # Threshold tuning for validation set
    if threshold is None:
        val_pred = model.predict(validation_input, verbose=1, batch_size=64)
        # print("val_pred start")
        # print(val_pred)
        # print("val_pred end")
        print(f'Prediction shape: {val_pred.shape}')
        best_threshold = 0.05
        best_fscore = 0.0
        for prob in np.arange(0.01, 0.99, 0.1):
            prediction_label = classify_by_threshold(val_pred, prob)
            # print("prediction_label start")
            # print(prediction_label)
            # print("prediction_label end") 
            avg_precision_0, avg_recall_0, avg_f1_0, avg_precision_1, avg_recall_1, avg_f1_1, _ = compute_performance(
                validation_output, prediction_label, val_texts, tokenizer, max_length, False
            )

            results += f"{count},{prob},{toxicClassifier.ALGO},{avg_precision_0},{avg_recall_0},{avg_f1_0},{avg_precision_1},{avg_recall_1},{avg_f1_1},{time_elapsed}\n"

            ######################################################
            # print(f"Precision non-toxic: {avg_precision_0}")
            # print(f"Recall non-toxic: {avg_recall_0}")
            # print(f"F1 non-toxic: {avg_f1_0}")
            # print(f"Precision toxic: {avg_precision_1}")
            # print(f"Recall toxic: {avg_recall_1}")
            # print(f"F1 toxic: {avg_f1_1}")
            ######################################################
            
            if avg_f1_1 > best_fscore:
                best_threshold = prob
                best_fscore = avg_f1_1
            print(f"Threshold: {prob}, F1 toxic: {avg_f1_1}")

        print(f"Best F1 toxic: {best_fscore} at threshold: {best_threshold}")

    # Evaluate on the test set using a fixed threshold
    else:
        prediction_label = classify_by_threshold(test_pred, threshold)
        avg_precision_0, avg_recall_0, avg_f1_0, avg_precision_1, avg_recall_1, avg_f1_1, retro_output = compute_performance(
            test_output, prediction_label, test_texts, tokenizer, max_length, True
        )
        error_analysis_output.append(retro_output)

        print(f"Precision non-toxic: {avg_precision_0}")
        print(f"Recall non-toxic: {avg_recall_0}")
        print(f"F1 non-toxic: {avg_f1_0}")
        print(f"Precision toxic: {avg_precision_1}")
        print(f"Recall toxic: {avg_recall_1}")
        print(f"F1 toxic: {avg_f1_1}")

        results += f"{count},{threshold},{toxicClassifier.ALGO},{avg_precision_0},{avg_recall_0},{avg_f1_0},{avg_precision_1},{avg_recall_1},{avg_f1_1},{time_elapsed}\n"

    count += 1

# Debugging: Final output
print("Cross-validation completed.")
print("Results Summary:\n", results)


Using split-1 as test data...


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['roberta.encoder.layer.2.output.LayerNorm.weight', 'roberta.encoder.layer.0.attention.self.key.weight', 'roberta.encoder.layer.9.attention.self.value.weight', 'roberta.encoder.layer.11.intermediate.dense.bias', 'roberta.encoder.layer.11.attention.self.key.bias', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.1.output.dense.bias', 'roberta.encoder.layer.10.attention.output.dense.weight', 'roberta.encoder.layer.0.attention.self.value.weight', 'roberta.encoder.layer.4.attention.self.key.bias', 'roberta.encoder.layer.8.output.dense.bias', 'roberta.encoder.layer.11.attention.output.dense.weight', 'roberta.encoder.layer.1.intermediate.dense.bias', 'roberta.encoder.layer.6.attention.self.query.bias', 'roberta.encoder.layer.7.attention.output.dense.weight', 'roberta.encoder.layer.7.output.dense.bias', 'roberta.embeddings.LayerNorm.weight', 'roberta.encoder.layer.0.attention.self.val

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_ids (InputLayer)    │ (None, 70)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ token_type_ids            │ (None, 70)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_mask            │ (None, 70)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ my_layer (MyLayer)        │ (None, 1)              │              0 │ input_ids[0][0],       │
│                           │                        │                │ token_type_ids[0][0],  │
│                           │                        │                │ attention_mask[0][0]   │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_ids (InputLayer)    │ (None, 70)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ token_type_ids            │ (None, 70)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_mask            │ (None, 70)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ my_layer_1 (MyLayer)      │ (None, 1)              │              0 │ input_ids[0][0],       │
│                           │                        │                │ token_type_ids[0][0],  │
│                           │                        │                │ attention_mask[0][0]   │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Training the model...
Epoch 1/30
811/811 ━━━━━━━━━━━━━━━━━━━━ 218s 235ms/step - loss: 0.0372 - val_loss: 0.0658
Epoch 2/30
105/811 ━━━━━━━━━━━━━━━━━━━━ 1:54 162ms/step - loss: 0.0166

In [ ]:
# spans : [10, 11, 12, 13, 14, 15]

# text : "yeah that sucked, fixed"

#is_toxic : sucked


